Conceptual

   1. What is the difference between train and test set?
   Training set is to build the model, and test set is to validate the data build.
   
  


   2.Why is it important to have a seperate data set for training?
   
      After the seperation we can better understand the model, and minimizing the overfit.
      It also serves as check on the overfit, give estimate of performance on an independant dataset.
   
  

3. What are some advantages and disadvantages of the KNN algorithm?
    
    advantages:
                   1. Training is fast
                   2. Learn complex target functions
                   3. Dont lose information.
    Disadvantages:
                   1. Slow at query time
                   2. Storage issues
                   3. easily fooled by irrelevant features and high dimensions.
                   
   

4. Explain in your own words the bias variance tradeoff.
    
        Variance: it can only simulate the things seen before, overfit.
        Bias: Ignores the dataset features.
  

5.  Do you think KNN suffers from high bias or high variance?
    
    
        Depends on the K, when K is small it is bias, when K is big, it is variance.
     

    

6.List some possible solutions when suffering from high variance.
    
    Get more data or decrease more features.
    
    
    

7.List some possible solutions when suffereing from high bias.
   
     1. find more features.
     

8.Would you choose K when doing K Nearest Neighbors?

 K fold validataion do that.


Part 2 :

1.Perform regression on the prostate data set. Your goal is to minimize the mean squared error (MSE). Split this into a several parts and at a minimum utilize

    Linear Regression
    KNN
    Lasso
    Ridge
    Use cross validation to tune the best model you can
    
    
    
   


In [36]:
from pandas import *

data = read_csv('https://raw.githubusercontent.com/galvin-mj/DAT_ATL_15/master/Datasets/ProstateCancer.csv')
del data['Unnamed: 0']
train = data[data['train']=='T']
test = data[data['train']=='F']
X_train = train[['lcavol','lweight','age','lbph','svi','gleason','pgg45']]
y_train = train[['lpsa']]
X_test = test[['lcavol','lweight','age','lbph','svi','gleason','pgg45']]
y_test = test[['lpsa']]


In [37]:
# Linear Regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
lr_rgr = linear_model.LinearRegression()
lr_rgr.fit(X_train,y_train)
print("coefficient {}".format(lr_rgr.coef_))
print("intercept {}".format(lr_rgr.intercept_))
predict = lr_rgr.predict(X_test)
LR_MSE = mean_squared_error(y_test, predict)
print('Mean squared error for liner regression: {}'.format(LR_MSE))

coefficient [[ 0.48908263  0.61223896 -0.01713236  0.15436282  0.55160293 -0.00848756
   0.00550287]]
intercept [ 0.47720966]
Mean squared error for liner regression: 0.461141049694


In [38]:
# PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import make_scorer, mean_squared_error

mse = make_scorer(mean_squared_error, greater_is_better = False)
poly = PolynomialFeatures(2)


#Polynomial
poly_rgr = linear_model.LinearRegression()
poly_rgr.fit(poly.fit_transform(X_train),y_train)
prediction = poly_rgr.predict(poly.fit_transform(X_test))
Poly_MSE = mean_squared_error(y_test, prediction )
print Poly_MSE

0.683398060975


In [46]:
poly3 = PolynomialFeatures(3)

#Polynomial
poly3_rgr = linear_model.LinearRegression()
poly3_rgr.fit(poly3.fit_transform(X_train),y_train)
Poly3_MSE = mean_squared_error(y_test, poly3_rgr.predict(poly3.fit_transform(X_test)))
print Poly3_MSE
                                                                             

733.236736432


In [40]:
#KNN
from sklearn import neighbors
from sklearn import grid_search
knn = neighbors.KNeighborsRegressor()
knn.fit(X_train,y_train)
knn_predict = knn.predict(X_test)
LR_MSE = mean_squared_error(y_test, knn_predict)
print('Mean squared error for liner regression: {}'.format(LR_MSE))
param_grid = [{'n_neighbors':[1,2,3,4,5,6,7,8,9,10]}]
knnrgr = grid_search.GridSearchCV(knn, param_grid)
knnrgr.fit(X_train,y_train)
print knnrgr.best_estimator_
print knnrgr.best_params_
print knnrgr.score
knnrgrpredict = knnrgr.predict(X_test)
LR_MSE2 = mean_squared_error(y_test, knnrgrpredict)
print('Mean squared error for liner regression: {}'.format(LR_MSE2))

Mean squared error for liner regression: 1.22325178741
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_neighbors=6, p=2, weights='uniform')
{'n_neighbors': 6}
<bound method GridSearchCV.score of GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_neighbors=5, p=2, weights='uniform'),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}],
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)>
Mean squared error for liner regression: 1.20669829128


In [41]:
#Ridge
from sklearn import preprocessing
from sklearn import grid_search
param_grid = [{'alpha':np.linspace(1e-8, 1, 3000)}]
X_train_scaled = preprocessing.scale(X_train)
y_train_scaled = preprocessing.scale(y_train)
X_test_scaled = preprocessing.scale(X_test)
y_test_scaled = preprocessing.scale(y_test)
#Ridge
ridge_rgr = linear_model.Ridge(normalize = True)
ridge_cv = grid_search.GridSearchCV(ridge_rgr, param_grid,cv=5)
ridge_cv.fit(X_train_scaled, y_train_scaled)

x_test_scaled_predict = ridge_cv.best_estimator_.predict(X_test_scaled)
Ridge_MSE = mean_squared_error(y_test_scaled,x_test_scaled_predict )
print Ridge_MSE

0.527053487027


In [42]:
#lasso
lasso_rgr = linear_model.Lasso(normalize = True)
lasso_cv = grid_search.GridSearchCV(lasso_rgr, param_grid,cv=5)
lasso_cv.fit(X_train_scaled, y_train_scaled)
x_tested_lassopredict = lasso_cv.best_estimator_.predict(preprocessing.scale(X_test))
Lasso_MSE = mean_squared_error(y_test_scaled,x_tested_lassopredict )
print Lasso_MSE

0.492411948696



2.#Where there any features that were highly correlated?



In [43]:
X_train.corr()

,lcavol,lweight,age,lbph,svi,gleason,pgg45
lcavol,1.000000,0.300232,0.286324,0.063168,0.592949,0.426414,0.483161
lweight,0.300232,1.000000,0.316723,0.437042,0.181054,0.023558,0.074166
age,0.286324,0.316723,1.000000,0.287346,0.128902,0.365915,0.275806
lbph,0.063168,0.437042,0.287346,1.000000,-0.139147,0.032992,-0.030404
svi,0.592949,0.181054,0.128902,-0.139147,1.000000,0.306875,0.481358
gleason,0.426414,0.023558,0.365915,0.032992,0.306875,1.000000,0.757056
pgg45,0.483161,0.074166,0.275806,-0.030404,0.481358,0.757056,1.000000


gleason and pgg45 are highly correlated

3.What features were the most important? Unimportant?

In [44]:
print lr_rgr.coef_
feature_names= list(X_train.columns)
print feature_names


[[ 0.48908263  0.61223896 -0.01713236  0.15436282  0.55160293 -0.00848756
   0.00550287]]
['lcavol', 'lweight', 'age', 'lbph', 'svi', 'gleason', 'pgg45']


lcvol has the highest positive influence
age has the highest negative influence    they are the most inportanta
the most unimportatnt feature is gleason


4.What model worked best? What was the MSE of each?
the best is linear regression
MSE are:   linear 0.46114
           poly 0.68
           knn   1.2
           



In [ ]:
difficulty cant figure out whats wrong with poly(3)